# Casualties to Vehicles - UK Data

## Use Case:

With the new view titled stg.vehicles, analysts need the casualties per vehicle to be categorized as following:

Casuality Count and 'Bucket':

- 0 - None
- 1 - Single
- 2 to 4 - Multiple
- 5 to 10 - High Multiple
- Greater than 10 - Exteme Multiple

## Current State:

A view is available that contains the count of casualties per vehicle. The code to create the view is show below:  
```
USE [UK_Accidents_I]
GO


/*
Vehicles with casualties
-- Requirements:
  1) All fields from stg.Vehicles
  2) Count of Casualties in each vehicle "vehicle_casualty_count"
  3) A constant field (or flag) that show that this is a "casualty_vehicle" make it 1
*/
CREATE  OR ALTER VIEW [stg].[Vehicles_Enhanced] AS
SELECT veh.accident_index
	  ,veh.accident_reference
	  ,veh.vehicle_reference
	  ,veh.vehicle_type
	  ,veh.sex_of_driver
	  ,veh.age_band_of_driver
	  ,1 as 'casualty_vehicle'
	  ,veh.age_of_driver
	  ,count(*) as vehicle_casualty_count
FROM stg.Vehicles veh
	 INNER JOIN stg.Casualties cas
	 ON veh.accident_index = cas.accident_index
	    AND veh.accident_reference = cas.accident_reference
	    AND veh.vehicle_reference = cas.vehicle_reference
GROUP BY veh.accident_index
	  ,veh.accident_reference
	  ,veh.vehicle_reference
	  ,veh.vehicle_type
	  ,veh.sex_of_driver
	  ,veh.age_band_of_driver
	  ,veh.age_of_driver

UNION ALL

SELECT veh.accident_index
	  ,veh.accident_reference
	  ,veh.vehicle_reference
	  ,veh.vehicle_type
	  ,veh.sex_of_driver
	  ,veh.age_band_of_driver
	  ,0 as 'casualty_vehicle'
	  ,veh.age_of_driver
	  ,0 as vehicle_casualty_count
FROM stg.Vehicles veh
LEFT OUTER JOIN (SELECT DISTINCT veh.*
				 FROM stg.Vehicles veh
				 INNER JOIN stg.Casualties cas
					ON veh.accident_index = cas.accident_index
					AND veh.accident_reference = cas.accident_reference
					AND veh.vehicle_reference = cas.vehicle_reference) vwc
					ON veh.accident_index = vwc.accident_index
					AND veh.vehicle_reference = vwc.vehicle_reference
					WHERE vwc.accident_index is null
;


GO
```

## Solution:

Use a SWITCH statement in PowerPivot (DAX) to create the required field / feature. The field / feature name will be _casualty\_to\_vehicle\_classification_.

The switch statement for _casualty\_to\_vehicle\_classification_ is as follows:

```
casualty_to_vehicle_classification:=SWITCH(TRUE(),qCasualtyToVehicle[vehicle_casualty_count] = 0, "None"
		      ,qCasualtyToVehicle[vehicle_casualty_count] = 1, "Single"
		      ,qCasualtyToVehicle[vehicle_casualty_count] <= 4, "Multiple"
		      ,qCasualtyToVehicle[vehicle_casualty_count] <= 10, "High Multiple"
		      ,qCasualtyToVehicle[vehicle_casualty_count] > 10, "Extreme Multiple"
		      , "Error")
```